In [1]:
import numpy as np

opcodeInput=np.loadtxt("input_day19.txt",delimiter=",",dtype=np.int64)

class OpCodeManager:
    def __init__(self, opcode):
        self.opcode = opcode
        self.relativeBaseIdx=0
    def __getitem__(self, idx):
        if idx >= self.opcode.size:
            self.opcode.resize(idx+1, refcheck=False) # fill rest with zeros
        return self.opcode[idx]
    def __setitem__(self, idx, value):
        if idx >= self.opcode.size:
            self.opcode.resize(idx+1, refcheck=False) # fill rest with zeros
        self.opcode[idx] = value

def decodeCommand(command):
    code=command%100
    p1Mode=int(command/100)%10
    p2Mode=int(command/1000)%10
    p3Mode=int(command/10000)%10
    return [code, p1Mode, p2Mode, p3Mode]

def getArgument(opcode, mode, parameter):
    if mode == 0:
        # position mode
        return opcode[parameter]
    elif mode == 1:
        # immediate mode
        return parameter
    else:
        # relative mode
        return opcode[opcode.relativeBaseIdx+parameter]
    
def getWritePosition(opcode, mode, parameter):
    if mode == 0:
        # position mode
        return parameter
    elif mode == 1:
        # immediate mode
        print("ERROR! Immediate mode not intended for writing")
        print(mode, parameter, opcode)
    else:
        # relative mode
        return opcode.relativeBaseIdx+parameter

In [13]:
class Computer:
    def __init__(self, opcodeMgr):
        self.opcode = opcodeMgr
        self.outputList=[]
        self.input=0
        
    def clearOutput(self):
        self.outputList=[]
    
    def visualize(self):
        return ''
    
    def printOutputList(self):
        outputStr=''
        for elem in self.outputList:
            outputStr += chr(elem)
        print(outputStr)
    
    def run(self, idx):
    
        while True:

            decoded = decodeCommand(self.opcode[idx])
  
            if decoded[0] == 99:
                #print("Finished!")
                return 0
            elif decoded[0] == 1:
                #add p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                self.opcode[arg3] = arg1 + arg2
                idx+=4
            elif decoded[0] == 2:
                #multiply p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                self.opcode[arg3] = arg1 * arg2
                idx+=4
            elif decoded[0] == 3:
                #input p1
                p1 = self.opcode[idx+1]
                arg1 = getWritePosition(self.opcode,decoded[1],p1)
                # break here and wait for updated input from extern
                #print("Waiting for input")
                yield 1
                var = self.input
                self.opcode[arg1] = np.int64(var)
                idx+=2
            elif decoded[0] == 4:
                #output p1
                p1 = self.opcode[idx+1]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                #print("Output: ", arg1)
                self.outputList.append(arg1)
                idx+=2
            elif decoded[0] == 5:
                #jump-if-true p1 p2
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                if arg1 != 0:
                    idx=arg2
                else:      
                    idx=idx+3
            elif decoded[0] == 6:
                #jump-if-false p1 p2
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                if arg1 == 0:
                    idx=arg2
                else:      
                    idx+=3
            elif decoded[0] == 7:
                #less-than p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                if arg1 < arg2:
                    self.opcode[arg3]=1
                else:      
                    self.opcode[arg3]=0
                idx+=4
            elif decoded[0] == 8:
                #equals p1 p2 p3
                p1 = self.opcode[idx+1]
                p2 = self.opcode[idx+2]
                p3 = self.opcode[idx+3]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                arg2 = getArgument(self.opcode,decoded[2],p2)
                arg3 = getWritePosition(self.opcode,decoded[3],p3)
                if arg1 == arg2:
                    self.opcode[arg3]=1
                else:      
                    self.opcode[arg3]=0
                idx+=4
            elif decoded[0] == 9:
                # change relative base p1
                p1 = self.opcode[idx+1]
                arg1 = getArgument(self.opcode,decoded[1],p1)
                self.opcode.relativeBaseIdx += arg1
                idx+=2
            else:
                print("Invalid code ", self.opcode, idx, self.opcode[idx], decoded)
                return

In [25]:
#task1
affectedCoords = 0
outputStr=''
for y in range(55):
    for x in range(51):
        inputCoords = (x,y)
        opcodeMgr=OpCodeManager(opcodeInput.copy())
        computer = Computer(opcodeMgr)
        for i,_ in enumerate(computer.run(0)):
            computer.input=inputCoords[i]
        if computer.outputList[0] == 1:
            affectedCoords += 1
            outputStr+='#'
        else:
            outputStr+='.'
    outputStr+='\n'
            
            
print(affectedCoords)

print(outputStr)


192
#..................................................
...................................................
.#.................................................
...................................................
..#................................................
...#...............................................
...#...............................................
....#..............................................
....#..............................................
.....#.............................................
.....##............................................
......#............................................
......##...........................................
.......##..........................................
.......##..........................................
........##.........................................
........##.........................................
.........##........................................
.........###.......................................
........

In [26]:
# task 2

def isAttracted(opcodeInput,inputTuple):
    opcodeMgr=OpCodeManager(opcodeInput.copy())
    computer = Computer(opcodeMgr)
    for i,_ in enumerate(computer.run(0)):
        computer.input=inputTuple[i]
    if computer.outputList[0] == 1:
        return True
    else:
        return False
    
def findClosestCoord(opcodeInput):
    y = 50
    xMin = y
    xMax = 0
    for x in range(0,y):
        if isAttracted(opcodeInput,(x,y)):
            xMin = min(x,xMin)
            xMax = max(x,xMax)

    xSizeShip=100
    ySizeShip=100
    while True:
        print(y,xMin,xMax)
        # find new smallest attracting x index for current row
        while not isAttracted(opcodeInput,(xMin,y)):
            xMin += 1

        # find new largest attracting x index for current row
        while isAttracted(opcodeInput,(xMax+1,y)):
            xMax += 1
                
        print(xMin,xMax)

        # check if the ship will fit:
        if xMax - xMin > xSizeShip:
            # would fit in row
            for x in range(xMin,xMax+1-(xSizeShip-1)):
                if isAttracted(opcodeInput,(x,y+(ySizeShip-1))):
                    print("Found it! Left corner is at ", (x,y), x*10000+y)
                    return

        y += 1
        
findClosestCoord(opcodeInput)

50 25 30
25 30
51 25 30
25 31
52 25 31
26 32
53 26 32
26 32
54 26 32
27 33
55 27 33
27 33
56 27 33
28 34
57 28 34
28 35
58 28 35
29 35
59 29 35
29 36
60 29 36
30 36
61 30 36
30 37
62 30 37
31 38
63 31 38
31 38
64 31 38
32 39
65 32 39
32 40
66 32 40
33 40
67 33 40
33 41
68 33 41
34 41
69 34 41
34 42
70 34 42
35 43
71 35 43
35 43
72 35 43
36 44
73 36 44
36 44
74 36 44
37 45
75 37 45
37 46
76 37 46
38 46
77 38 46
38 47
78 38 47
39 48
79 39 48
39 48
80 39 48
40 49
81 40 49
40 49
82 40 49
41 50
83 41 50
41 51
84 41 51
42 51
85 42 51
42 52
86 42 52
43 53
87 43 53
43 53
88 43 53
44 54
89 44 54
44 54
90 44 54
45 55
91 45 55
45 56
92 45 56
46 56
93 46 56
46 57
94 46 57
46 57
95 46 57
47 58
96 47 58
47 59
97 47 59
48 59
98 48 59
48 60
99 48 60
49 61
100 49 61
49 61
101 49 61
50 62
102 50 62
50 62
103 50 62
51 63
104 51 63
51 64
105 51 64
52 64
106 52 64
52 65
107 52 65
53 65
108 53 65
53 66
109 53 66
54 67
110 54 67
54 67
111 54 67
55 68
112 55 68
55 69
113 55 69
56 69
114 56 69
56 70
115 56 70


498 244 306
244 306
499 244 306
245 307
500 245 307
245 308
501 245 308
246 308
502 246 308
246 309
503 246 309
247 310
504 247 310
247 310
505 247 310
248 311
506 248 311
248 311
507 248 311
249 312
508 249 312
249 313
509 249 313
250 313
510 250 313
250 314
511 250 314
251 314
512 251 314
251 315
513 251 315
252 316
514 252 316
252 316
515 252 316
253 317
516 253 317
253 318
517 253 318
253 318
518 253 318
254 319
519 254 319
254 319
520 254 319
255 320
521 255 320
255 321
522 255 321
256 321
523 256 321
256 322
524 256 322
257 322
525 257 322
257 323
526 257 323
258 324
527 258 324
258 324
528 258 324
259 325
529 259 325
259 326
530 259 326
260 326
531 260 326
260 327
532 260 327
261 327
533 261 327
261 328
534 261 328
262 329
535 262 329
262 329
536 262 329
263 330
537 263 330
263 330
538 263 330
264 331
539 264 331
264 332
540 264 332
265 332
541 265 332
265 333
542 265 333
266 334
543 266 334
266 334
544 266 334
267 335
545 267 335
267 335
546 267 335
268 336
547 268 336
268 337


913 447 562
447 562
914 447 562
448 563
915 448 563
448 563
916 448 563
449 564
917 449 564
449 565
918 449 565
450 565
919 450 565
450 566
920 450 566
451 567
921 451 567
451 567
922 451 567
452 568
923 452 568
452 568
924 452 568
453 569
925 453 569
453 570
926 453 570
454 570
927 454 570
454 571
928 454 571
455 571
929 455 571
455 572
930 455 572
456 573
931 456 573
456 573
932 456 573
457 574
933 457 574
457 575
934 457 575
458 575
935 458 575
458 576
936 458 576
459 576
937 459 576
459 577
938 459 577
459 578
939 459 578
460 578
940 460 578
460 579
941 460 579
461 579
942 461 579
461 580
943 461 580
462 581
944 462 581
462 581
945 462 581
463 582
946 463 582
463 583
947 463 583
464 583
948 464 583
464 584
949 464 584
465 584
950 465 584
465 585
951 465 585
466 586
952 466 586
466 586
953 466 586
467 587
954 467 587
467 587
955 467 587
468 588
956 468 588
468 589
957 468 589
469 589
958 469 589
469 590
959 469 590
470 591
960 470 591
470 591
961 470 591
471 592
962 471 592
471 592
